In [12]:
import pandas as pd
from pyspark.sql import SparkSession
import numpy as np

In [2]:
spark = (SparkSession.builder.appName('data_import')
            .config("spark.executor.memory", "4g") 
            .config("spark.executor.cores", "4") 
            .config("spark.executor.instances", "2")
            .config("spark.driver.memory", "2g") 
            .config("spark.sql.shuffle.partitions", "200")
            .config("spark.default.parallelism", "200")
            .getOrCreate())

23/12/23 23:21:51 WARN Utils: Your hostname, Gears-WindowsPC.local resolves to a loopback address: 127.0.0.1; using 10.0.0.126 instead (on interface en0)
23/12/23 23:21:51 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/23 23:21:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/12/23 23:21:53 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


# Patients Import 

In [3]:
spark

In [4]:
patient_df = pd.read_parquet('patients/')

In [18]:
patient_df['weight'] = np.where(patient_df['weight'] == '?', None, patient_df['weight'])

In [51]:
patient_df['race'] = np.where(patient_df['race'] == '?', None, patient_df['race'])

/var/folders/_s/6p3493gn3cl7krkpzh8cq_700000gn/T/ipykernel_12368/3439493894.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  patient_df['race'] = np.where(patient_df['race'] == '?', None, patient_df['race'])


In [52]:
patient_df['gender'] = np.where(patient_df['gender'] == '?', None, patient_df['gender'])

/var/folders/_s/6p3493gn3cl7krkpzh8cq_700000gn/T/ipykernel_12368/3057950818.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  patient_df['gender'] = np.where(patient_df['gender'] == '?', None, patient_df['gender'])


In [28]:
patient_df.rename(columns={'patient_id': 'id', 'patient_name': 'name'},inplace=True)

In [21]:
database_name = 'hospital_readmissions_prod'
table_name = 'patients'
db_user = 'postgres'
db_password = 'postgres'
db_host = 'localhost'
db_port = '5432'

In [22]:
from sqlalchemy import create_engine
import psycopg2

In [42]:
conn = psycopg2.connect(dbname=database_name, user=db_user, password=db_password, host=db_host, port=db_port)

In [43]:
# Create a SQLAlchemy engine
engine = create_engine(f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{database_name}')

In [39]:
pd.read_sql('select * from patients', conn)

/var/folders/_s/6p3493gn3cl7krkpzh8cq_700000gn/T/ipykernel_12368/1160372290.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql('select * from patients', conn)


,id,name,gender,race,age,weight


In [41]:
# conn.close()

In [ ]:
patient_df.drop_duplicates(subset=)

In [55]:
patient_df = patient_df.drop_duplicates(subset=['id', 'name'])

In [92]:
# Write the Pandas DataFrame to the PostgreSQL table
patient_df.to_sql(table_name, engine, if_exists='append', index=False)

518

In [118]:
patient_df.to_csv('patients.csv', index=False)

# Physicians 

In [63]:
physicians_df = pd.read_parquet('physicians')

In [64]:
physicians_df.head()

,id,physician_specialty
0,1,AllergyandImmunology
1,2,Anesthesiology
2,3,Anesthesiology-Pediatric
3,4,Cardiology
4,5,Cardiology-Pediatric


In [68]:
physicians_df = physicians_df.rename(columns={'physician_specialty':'specialty'})

In [74]:
def import_data_to_postgres(table_name: str, df: pd.DataFrame, engine, if_exists='append'):
    df.to_sql(table_name, engine, if_exists=if_exists, index=False)
    return f'{table_name} has imported data successfully'

In [93]:
import_data_to_postgres(table_name='physicians', df=physicians_df, engine=engine)

'physicians has imported data successfully'

In [119]:
physicians_df.to_csv('physicians.csv', index=False)

# Medications 

In [75]:
medications_df = pd.read_parquet('medications/')

In [76]:
medications_df.head()

,id,medication_name
0,1,acarbose
1,2,acetohexamide
2,3,chlorpropamide
3,4,citoglipton
4,5,examide


In [94]:
import_data_to_postgres('medications', medications_df, engine)

'medications has imported data successfully'

In [120]:
medications_df.to_csv('medications.csv')

# Insurance 

In [86]:
insurance_df = pd.read_parquet('insurances/')

In [88]:
insurance_df.head()

,id,insurance_cd,insurance_name
0,1,MC,Medicare
1,2,MD,Medicaid
2,3,HM,Health Maintenance Organization (HMO)
3,4,UN,Unknown
4,5,BC,Blue Cross Blue Shield


In [95]:
import_data_to_postgres('insurances', insurance_df, engine)

'insurances has imported data successfully'

In [121]:
insurance_df.to_csv('insurances.csv', index=False)

# Diagnosis 

In [90]:
diagnosis_df = pd.read_parquet('diagnoses/')

In [91]:
diagnosis_df.head()

,diag,diag_description
0,514,Pulmonary congestion and hypostasis
1,682,Other cellulitis and abscess
2,V55,Attention to artificial openings
3,V58,Encounter for other and unspecified procedures...
4,174,Malignant neoplasm of female breast


In [96]:
import_data_to_postgres('diagnoses', diagnosis_df, engine)

'diagnoses has imported data successfully'

In [122]:
diagnosis_df.to_csv('diagnoses.csv', index=False)

# Lab Results 

In [97]:
lab_results_df = pd.read_parquet('lab_results/')

In [98]:
lab_results_df.head()

,admission_id,max_glu_serum,A1Cresult
0,2278392,None,None
1,149190,None,None
2,64410,None,None
3,500364,None,None
4,16680,None,None


In [212]:
lab_results_df = lab_results_df.rename(columns={'A1Cresult':'a1cresult'})

In [213]:
import_data_to_postgres('lab_results', lab_results_df, engine)

'lab_results has imported data successfully'

In [215]:
lab_results_df.shape

(101766, 3)

In [217]:
lab_results_df.drop_duplicates().shape

(101766, 3)

In [214]:
lab_results_df.to_csv('csv_files/lab_results.csv', index=False)

# Admissions 

In [99]:
admissions_df = pd.read_parquet('admissions/')

In [113]:
admissions_df = admissions_df.rename(columns={'admission_id': 'id'})

In [111]:
admissions_df = admissions_df.drop(columns='payer_code')

In [124]:
admissions_df.to_csv('admissions.csv', index=False)

In [115]:
import_data_to_postgres('admissions', admissions_df, engine)

'admissions has imported data successfully'

# Medication Changes 

In [78]:
medication_changes = pd.read_parquet('medication_changes/')

In [82]:
medication_changes = medication_changes[~(medication_changes['medication_id']=='N/A')]

In [116]:
medication_changes.head()

,admission_id,medication_id,change_status
1,2278392,1,No
2,2278392,2,No
3,2278392,3,No
4,2278392,4,No
5,2278392,5,No


In [125]:
medication_changes.to_csv('medication_changes.csv', index=False)

23/12/24 01:17:04 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 199815 ms exceeds timeout 120000 ms
23/12/24 01:17:04 WARN SparkContext: Killing executors is not supported by current scheduler.
23/12/24 01:17:04 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$

23/12/24 01:17:14 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:101)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:85)
	at org.apache.spark.storage.BlockManagerMaster.registerBlockManager(BlockManagerMaster.scala:80)
	at org.apache.spark.storage.BlockManager.reregister(BlockManager.scala:642)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:1223)
	at org.apache.spark.executor.Executor.$anonfun$heartbeater$1(Executor.scala:295)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at org.apache.spark.util.Utils$.logUncaughtExceptions(Utils.scala:1928)

23/12/24 01:17:24 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:101)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:85)
	at org.apache.spark.storage.BlockManagerMaster.registerBlockManager(BlockManagerMaster.scala:80)
	at org.apache.spark.storage.BlockManager.reregister(BlockManager.scala:642)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:1223)
	at org.apache.spark.executor.Executor.$anonfun$heartbeater$1(Executor.scala:295)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at org.apache.spark.util.Utils$.logUncaughtExceptions(Utils.scala:1928)

23/12/24 01:17:34 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$driverEndpoint(BlockManagerMasterEndpoint.scala:123)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.isExecutorAlive$lzycompute$1(BlockManagerMasterEndpoint.scala:688)
	at org.apache.spark.storage.BlockManagerMasterE

23/12/24 01:17:44 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$driverEndpoint(BlockManagerMasterEndpoint.scala:123)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.isExecutorAlive$lzycompute$1(BlockManagerMasterEndpoint.scala:688)
	at org.apache.spark.storage.BlockManagerMasterE

...

23/12/24 01:30:51 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$driverEndpoint(BlockManagerMasterEndpoint.scala:123)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.isExecutorAlive$lzycompute$1(BlockManagerMasterEndpoint.scala:688)
	at org.apache.spark.storage.BlockManagerMasterE

23/12/24 01:31:01 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$driverEndpoint(BlockManagerMasterEndpoint.scala:123)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.isExecutorAlive$lzycompute$1(BlockManagerMasterEndpoint.scala:688)
	at org.apache.spark.storage.BlockManagerMasterE

23/12/24 01:33:23 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$driverEndpoint(BlockManagerMasterEndpoint.scala:123)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.isExecutorAlive$lzycompute$1(BlockManagerMasterEndpoint.scala:688)
	at org.apache.spark.storage.BlockManagerMasterE

23/12/24 01:33:43 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:101)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:85)
	at org.apache.spark.storage.BlockManagerMaster.registerBlockManager(BlockManagerMaster.scala:80)
	at org.apache.spark.storage.BlockManager.reregister(BlockManager.scala:642)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:1223)
	at org.apache.spark.executor.Executor.$anonfun$heartbeater$1(Executor.scala:295)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at org.apache.spark.util.Utils$.logUncaughtExceptions(Utils.scala:1928)

23/12/24 01:33:53 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$driverEndpoint(BlockManagerMasterEndpoint.scala:123)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.isExecutorAlive$lzycompute$1(BlockManagerMasterEndpoint.scala:688)
	at org.apache.spark.storage.BlockManagerMasterE

In [117]:
import_data_to_postgres('medication_changes', medication_changes, engine)

'medication_changes has imported data successfully'

# Admissions Physicians

In [130]:
admissions_full_df = pd.read_csv('csv_files/diabetic_data.csv')

In [131]:
admissions_full_df.head()

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),?,6,25,1,1,...,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),?,1,1,7,3,...,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,[20-30),?,1,1,7,2,...,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,[30-40),?,1,1,7,2,...,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,[40-50),?,1,1,7,1,...,No,Steady,No,No,No,No,No,Ch,Yes,NO


In [132]:
admissions_full_df.columns

Index(['encounter_id', 'patient_nbr', 'race', 'gender', 'age', 'weight',
       'admission_type_id', 'discharge_disposition_id', 'admission_source_id',
       'time_in_hospital', 'payer_code', 'medical_specialty',
       'num_lab_procedures', 'num_procedures', 'num_medications',
       'number_outpatient', 'number_emergency', 'number_inpatient', 'diag_1',
       'diag_2', 'diag_3', 'number_diagnoses', 'max_glu_serum', 'A1Cresult',
       'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide',
       'glimepiride', 'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide',
       'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone',
       'tolazamide', 'examide', 'citoglipton', 'insulin',
       'glyburide-metformin', 'glipizide-metformin',
       'glimepiride-pioglitazone', 'metformin-rosiglitazone',
       'metformin-pioglitazone', 'change', 'diabetesMed', 'readmitted'],
      dtype='object')

In [133]:
physicians_df.head()

,id,specialty
0,1,AllergyandImmunology
1,2,Anesthesiology
2,3,Anesthesiology-Pediatric
3,4,Cardiology
4,5,Cardiology-Pediatric


In [134]:
admissions_full_df.merge(physicians_df, how='left', left_on='medical_specialty', right_on='specialty')

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted,id,specialty
0,2278392,8222157,Caucasian,Female,[0-10),?,6,25,1,1,...,No,No,No,No,No,No,No,NO,38.0,Pediatrics-Endocrinology
1,149190,55629189,Caucasian,Female,[10-20),?,1,1,7,3,...,No,No,No,No,No,Ch,Yes,>30,NaN,NaN
2,64410,86047875,AfricanAmerican,Female,[20-30),?,1,1,7,2,...,No,No,No,No,No,No,Yes,NO,NaN,NaN
3,500364,82442376,Caucasian,Male,[30-40),?,1,1,7,2,...,No,No,No,No,No,Ch,Yes,NO,NaN,NaN
4,16680,42519267,Caucasian,Male,[40-50),?,1,1,7,1,...,No,No,No,No,No,Ch,Yes,NO,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101761,443847548,100162476,AfricanAmerican,Male,[70-80),?,1,3,7,3,...,No,No,No,No,No,Ch,Yes,>30,NaN,NaN
101762,443847782,74694222,AfricanAmerican,Female,[80-90),?,1,4,5,5,...,No,No,No,No,No,No,Yes,NO,NaN,NaN
101763,443854148,41088789,Caucasian,Male,[70-80),?,1,1,7,1,...,No,No,No,No,No,Ch,Yes,NO,NaN,NaN
101764,443857166,31693671,Caucasian,Female,[80-90),?,2,3,7,10,...,No,No,No,No,No,Ch,Yes,NO,63.0,Surgery-General


In [135]:
admissions_physicians = admissions_full_df.merge(physicians_df, how='left', left_on='medical_specialty', right_on='specialty')[['encounter_id', 'id']]

In [137]:
admissions_physicians = admissions_physicians.rename(columns={'encounter_id': 'admission_id'})

In [139]:
admissions_physicians = admissions_physicians.rename(columns={'id':'physician_id'})

In [140]:
import_data_to_postgres('admissions_physicians', admissions_physicians, engine)

'admissions_physicians has imported data successfully'

In [141]:
admissions_physicians.to_csv('admissions_physicians.csv', index=False)

# Readmission 

In [142]:
readmissions = pd.read_parquet('parquet_files/readmissions/')

In [143]:
readmissions.head()

,admission_id,patient_id,status
0,2278392,8222157,NO
1,149190,55629189,>30
2,64410,86047875,NO
3,500364,82442376,NO
4,16680,42519267,NO


In [144]:
import_data_to_postgres('readmissions', readmissions, engine)

'readmissions has imported data successfully'

In [145]:
readmissions.to_csv('csv_files/readmissions.csv', index=False)

# Admissions_diagnosis 

In [146]:
admissions_full_df

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),?,6,25,1,1,...,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),?,1,1,7,3,...,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,[20-30),?,1,1,7,2,...,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,[30-40),?,1,1,7,2,...,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,[40-50),?,1,1,7,1,...,No,Steady,No,No,No,No,No,Ch,Yes,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101761,443847548,100162476,AfricanAmerican,Male,[70-80),?,1,3,7,3,...,No,Down,No,No,No,No,No,Ch,Yes,>30
101762,443847782,74694222,AfricanAmerican,Female,[80-90),?,1,4,5,5,...,No,Steady,No,No,No,No,No,No,Yes,NO
101763,443854148,41088789,Caucasian,Male,[70-80),?,1,1,7,1,...,No,Down,No,No,No,No,No,Ch,Yes,NO
101764,443857166,31693671,Caucasian,Female,[80-90),?,2,3,7,10,...,No,Up,No,No,No,No,No,Ch,Yes,NO


In [150]:
admission_diagnosis = pd.melt(admissions_full_df, id_vars=['encounter_id'], value_vars=['diag_1', 'diag_2','diag_3'], var_name='diag_name', value_name='diag')[['encounter_id', 'diag']]


In [152]:
admission_diagnosis = admission_diagnosis.rename(columns={'encounter_id': 'admission_id'})

In [153]:
admission_diagnosis.head()

,admission_id,diag
0,2278392,250.83
1,149190,276
2,64410,648
3,500364,8
4,16680,197


In [160]:
diagnoses_df = pd.read_csv('csv_files/diagnoses.csv')

In [161]:
diagnoses_df

,diag,diag_description
0,514,Pulmonary congestion and hypostasis
1,682,Other cellulitis and abscess
2,V55,Attention to artificial openings
3,V58,Encounter for other and unspecified procedures...
4,174,Malignant neoplasm of female breast
...,...,...
911,243,Congenital hypothyroidism
912,E892,Conflagration not in building or structure
913,877,Open wound of buttock
914,876,Open wound of back


In [163]:
diagnoses_df = pd.read_sql('select * from diagnoses', conn)

/var/folders/_s/6p3493gn3cl7krkpzh8cq_700000gn/T/ipykernel_12368/4044889991.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  diagnoses_df = pd.read_sql('select * from diagnoses', conn)


In [164]:
diagnoses_df

,id,diag,diag_description
0,1,514,Pulmonary congestion and hypostasis
1,2,682,Other cellulitis and abscess
2,3,V55,Attention to artificial openings
3,4,V58,Encounter for other and unspecified procedures...
4,5,174,Malignant neoplasm of female breast
...,...,...,...
911,912,243,Congenital hypothyroidism
912,913,E892,Conflagration not in building or structure
913,914,877,Open wound of buttock
914,915,876,Open wound of back


In [166]:
admission_diagnosis = admission_diagnosis.merge(diagnoses_df, how='inner', left_on='diag', right_on='diag')[['admission_id', 'id']]

In [168]:
admission_diagnosis = admission_diagnosis.rename(columns={'id': 'diagnosis_id'})

In [169]:
import_data_to_postgres('admissions_diagnosis', admission_diagnosis, engine)

'admissions_diagnosis has imported data successfully'

In [170]:
admission_diagnosis.to_csv('admissions_diagnosis.csv', index=False)

# Admissions Procedures 

In [172]:
admissions_full_df.columns

Index(['encounter_id', 'patient_nbr', 'race', 'gender', 'age', 'weight',
       'admission_type_id', 'discharge_disposition_id', 'admission_source_id',
       'time_in_hospital', 'payer_code', 'medical_specialty',
       'num_lab_procedures', 'num_procedures', 'num_medications',
       'number_outpatient', 'number_emergency', 'number_inpatient', 'diag_1',
       'diag_2', 'diag_3', 'number_diagnoses', 'max_glu_serum', 'A1Cresult',
       'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide',
       'glimepiride', 'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide',
       'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone',
       'tolazamide', 'examide', 'citoglipton', 'insulin',
       'glyburide-metformin', 'glipizide-metformin',
       'glimepiride-pioglitazone', 'metformin-rosiglitazone',
       'metformin-pioglitazone', 'change', 'diabetesMed', 'readmitted'],
      dtype='object')

In [174]:
admissions_procedures = admissions_full_df[['encounter_id', 'num_lab_procedures', 'num_procedures', 'num_medications',
       'number_outpatient', 'number_emergency', 'number_inpatient', 'number_diagnoses']]

In [175]:
admissions_procedures = admissions_procedures.rename(columns={'encounter_id':'admission_id'})

In [176]:
admissions_procedures

,admission_id,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses
0,2278392,41,0,1,0,0,0,1
1,149190,59,0,18,0,0,0,9
2,64410,11,5,13,2,0,1,6
3,500364,44,1,16,0,0,0,7
4,16680,51,0,8,0,0,0,5
...,...,...,...,...,...,...,...,...
101761,443847548,51,0,16,0,0,0,9
101762,443847782,33,3,18,0,0,1,9
101763,443854148,53,0,9,1,0,0,13
101764,443857166,45,2,21,0,0,1,9


In [178]:
admissions_procedures = admissions_procedures.rename(columns={'num_lab_procedures': 'lab_procedures_cnt',
 'num_procedures': 'procedures_cnt',
 'num_medications': 'medications_cnt',
 'number_diagnoses': 'diagnoses_given_cnt',
 'number_outpatient': 'prev_outpatient_cnt',
 'number_inpatient': 'prev_inpatient_cnt',
 'number_emergency': 'prev_emergency_cnt'})

In [179]:
import_data_to_postgres('admissions_procedures', admissions_procedures, engine)

'admissions_procedures has imported data successfully'

In [180]:
admissions_procedures.to_csv('csv_files/admissions_procedures.csv', index=False)

# Admission Type 

In [184]:
df = pd.read_csv('csv_files/IDS_mapping.csv', header=None, skip_blank_lines=False, keep_default_na=False)

# Identify line breakers (',') in the DataFrame
line_breakers = df[df[0] == ""].index.tolist()

In [185]:
df

,0,1
0,admission_type_id,description
1,1,Emergency
2,2,Urgent
3,3,Elective
4,4,Newborn
...,...,...
63,22,Transfer from hospital inpt/same fac reslt in...
64,23,Born inside this hospital
65,24,Born outside this hospital
66,25,Transfer from Ambulatory Surgery Center


In [186]:
line_breakers

[9, 41]

In [187]:
# Separate the DataFrames based on line breakers (',')
df1 = df.iloc[:line_breakers[0], :].dropna(axis=1)
df2 = df.iloc[line_breakers[0]+1:line_breakers[1], :].dropna(axis=1)
df3 = df.iloc[line_breakers[1]+1:, :].dropna(axis=1)

In [189]:
# Use the first row as headers
df1.columns = df1.iloc[0]
df2.columns = df2.iloc[0]
df3.columns = df3.iloc[0]

# Drop the first row after using it as headers
df1 = df1.iloc[1:]
df2 = df2.iloc[1:]
df3 = df3.iloc[1:]

In [191]:
admission_types = df1

In [200]:
admission_types = admission_types.rename(columns={'admission_type_id': 'id'})

In [193]:
discharge_dispositions = df2

In [202]:
discharge_dispositions = discharge_dispositions.rename(columns={'discharge_disposition_id': 'id'})

In [195]:
admission_sources = df3

In [203]:
admission_sources = admission_sources.rename(columns={'admission_source_id': 'id'})

In [204]:
import_data_to_postgres('admission_sources', admission_sources, engine)

'admission_sources has imported data successfully'

In [205]:
import_data_to_postgres('admission_types', admission_types, engine)

'admission_types has imported data successfully'

In [206]:
import_data_to_postgres('discharge_dispositions', discharge_dispositions, engine)

'discharge_dispositions has imported data successfully'

In [207]:
admission_sources.to_csv('csv_files/admission_sources.csv', index=False)

In [208]:
discharge_dispositions.to_csv('discharge_dispositions.csv', index=False)

In [210]:
admission_types.to_csv('csv_files/admission_types.csv', index=False)